# Analisis Sentimen pada Ulasan Pengguna Block Blast! di Google Play Store

## Import Libraries

In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.8 MB/s eta 0:00:00


In [2]:
!pip install joblib

In [4]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.4 MB/s eta 0:00:00


In [5]:
import os
import re
import csv
import nltk
import string
import joblib
import kagglehub
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.svm import SVC
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D

In [6]:
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
!pip freeze > requirements.txt

## Loading Dataset

In [8]:
df = pd.read_csv("/content/scrapped_data_bb.csv")

In [9]:
jumlah_ulasan, jumlah_kolom = df.shape

In [10]:
df.head()

,userName,score,at,content
0,Pengguna Google,5,2025-04-12 04:08:29,"game kasual seperti ini memang menyenangkan,ta..."
1,Pengguna Google,5,2025-03-30 14:45:35,Game simple yang membuat ketagihan.Walau banya...
2,Pengguna Google,5,2025-04-09 03:55:48,jika kalian menemukan masalah blok tiba tiba d...
3,Pengguna Google,5,2025-04-13 14:30:55,"seruu bangett!!!, sampe ketagihan, ini juga ga..."
4,Pengguna Google,5,2025-03-21 05:22:14,sumpah ini game populer banget anak² satu desa...


In [11]:
df.columns

Index(['userName', 'score', 'at', 'content'], dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userName  20000 non-null  object
 1   score     20000 non-null  int64 
 2   at        20000 non-null  object
 3   content   19999 non-null  object
dtypes: int64(1), object(3)
memory usage: 625.1+ KB


In [13]:
# Membuat dataframe baru dan menghapus data bernilai NaN
clean_df = df.dropna()

In [14]:
# Menghapus baris duplikat
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

## Preprocessing Text

In [15]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(tokens):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    stemmed_words = [stemmer.stem(word) for word in tokens]
    return stemmed_words

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

Dari 20000 sampel data ulasan ini pasti banyak penggunaan kata slang bahasa Indonesia dan tidak bisa dicek satu per satu karena memakan waktu yang lama. Oleh karena itu saya menggunakan dataset slang words bahasa Indonesia dari kaggle. Link dataset dapat dilihat [di sini](https://www.kaggle.com/datasets/sodolanangbjkatio/slang-indonesia?resource=download).

In [16]:
path = kagglehub.dataset_download("sodolanangbjkatio/slang-indonesia")

print("Path to dataset files:", path)

100%|██████████| 6.72k/6.72k [00:00<00:00, 8.76MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sodolanangbjkatio/slang-indonesia/versions/1


In [17]:
file_path = os.path.join(path, "slang_indo.csv")

# Membuat dataframe untuk slangwords bahasa Indonesia
slangwords_df = pd.read_csv(file_path, header=None, names=["slang", "formal"])

slangwords = dict(zip(slangwords_df['slang'].str.lower(), slangwords_df['formal'].str.lower()))

In [18]:
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [19]:
tqdm.pandas()

# 1. Membersihkan teks
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# 2. Case folding
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# 3. Fix slangwords
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# 4. Tokenisasi
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# 5. Filtering stopwords
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# 6. Stemming
clean_df['text_stemming'] = clean_df['text_stopword'].progress_apply(stemmingText)

# 7. Gabung token jadi kalimat
clean_df['text_akhir'] = clean_df['text_stemming'].apply(toSentence)

# 8. Drop duplikat berdasarkan content
clean_df.drop_duplicates(subset=['content'], inplace=True)

  0%|          | 0/19999 [00:00<?, ?it/s]

In [20]:
# Hapus duplikat data setelah dibersihkan
clean_df.drop_duplicates(subset=['content'], inplace=True)

## Pelabelan

Untuk mempermudah pelabelan, saya menggunakan cara pelabelan berdasarkan score (rating aplikasi) sebagai berikut:


*   Score 1-2 = Negatif
*   Score 3 = Netral
*   Score 4-5 = Positif



In [21]:
def label_sentiment_from_score(score):
    if score >= 4:
        return 'Positif'
    elif score == 3:
        return 'Netral'
    else:
        return 'Negatif'

clean_df['sentiment_label'] = clean_df['score'].apply(label_sentiment_from_score)
print(clean_df['sentiment_label'].value_counts())

sentiment_label
Positif    15876
Negatif     2067
Netral       606
Name: count, dtype: int64


In [22]:
clean_df.head()

,userName,score,at,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_stemming,text_akhir,sentiment_label
0,Pengguna Google,5,2025-04-12 04:08:29,"game kasual seperti ini memang menyenangkan,ta...",game kasual seperti ini memang menyenangkantap...,game kasual seperti ini memang menyenangkantap...,game kasual seperti ini memang menyenangkantap...,"[game, kasual, seperti, ini, memang, menyenang...","[game, kasual, menyenangkantapi, masukan, pema...","[game, kasual, menyenangkantapi, masuk, main, ...",game kasual menyenangkantapi masuk main komuni...,Positif
1,Pengguna Google,5,2025-03-30 14:45:35,Game simple yang membuat ketagihan.Walau banya...,Game simple yang membuat ketagihanWalau banyak...,game simple yang membuat ketagihanwalau banyak...,game simple yang membuat ketagihanwalau banyak...,"[game, simple, yang, membuat, ketagihanwalau, ...","[game, simple, ketagihanwalau, game, konsep, g...","[game, simple, ketagihanwalau, game, konsep, g...",game simple ketagihanwalau game konsep game bl...,Positif
2,Pengguna Google,5,2025-04-09 03:55:48,jika kalian menemukan masalah blok tiba tiba d...,jika kalian menemukan masalah blok tiba tiba d...,jika kalian menemukan masalah blok tiba tiba d...,jika kalian menemukan masalah blok tiba tiba d...,"[jika, kalian, menemukan, masalah, blok, tiba,...","[menemukan, blok, kasih, jelek, pas, pasin, po...","[temu, blok, kasih, jelek, pas, pasin, pokok, ...",temu blok kasih jelek pas pasin pokok fokus bl...,Positif
3,Pengguna Google,5,2025-04-13 14:30:55,"seruu bangett!!!, sampe ketagihan, ini juga ga...",seruu bangett sampe ketagihan ini juga game ny...,seruu bangett sampe ketagihan ini juga game ny...,seruu sangat sampai ketagihan ini juga game n...,"[seruu, sangat, sampai, ketagihan, ini, juga, ...","[seruu, ketagihan, game, offline, diluar, puls...","[seruu, tagih, game, offline, luar, pulsa, mai...",seruu tagih game offline luar pulsa main ini p...,Positif
4,Pengguna Google,5,2025-03-21 05:22:14,sumpah ini game populer banget anak² satu desa...,sumpah ini game populer banget anak² satu desa...,sumpah ini game populer banget anak² satu desa...,sumpah ini game populer sekali anak² satu des...,"[sumpah, ini, game, populer, sekali, anak², sa...","[sumpah, game, populer, anak², desa, susah, si...","[sumpah, game, populer, anak, desa, susah, sin...",sumpah game populer anak desa susah sinyal ngu...,Positif


In [23]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18549 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   userName              18549 non-null  object
 1   score                 18549 non-null  int64 
 2   at                    18549 non-null  object
 3   content               18549 non-null  object
 4   text_clean            18549 non-null  object
 5   text_casefoldingText  18549 non-null  object
 6   text_slangwords       18549 non-null  object
 7   text_tokenizingText   18549 non-null  object
 8   text_stopword         18549 non-null  object
 9   text_stemming         18549 non-null  object
 10  text_akhir            18549 non-null  object
 11  sentiment_label       18549 non-null  object
dtypes: int64(1), object(11)
memory usage: 2.3+ MB


Setelah preprocessing text dan pelabelan, jumlah data sekitar 18549 sampel.

In [24]:
# Menyimpan hasil preprocessing ke file csv
clean_df.to_csv('clean_df.csv', index=False)

## Skema 1

* Pelatihan: Logistic Regression
* Ekstraksi Fitur: TF-IDF
* Pembagian Data: 80/20

In [25]:
# Label Encoding kolom sentiment_labeluntuk keperluan model logistic regression
le = LabelEncoder()
y_encoded = le.fit_transform(clean_df['sentiment_label'])

In [26]:
# Ekstraksi Fitur dengan TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(clean_df['text_akhir'])

In [27]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [28]:
# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression(max_iter=1000)

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.9129995282700991
Logistic Regression - accuracy_test: 0.8946091644204852


## Skema 2

* Pelatihan: Support Vector Machine (SVM)
* Ekstraksi Fitur: Bag of Words (BoW)
* Pembagian Data: 80/20

Label Encoding sudah dilakukan pada Skema 1. Label encoding ini juga membantu algoritma Naive Bayes yang membutuhkan target (y) dalam bentuk numerik supaya bisa menghitung probabilitas untuk tiap kelas.

In [29]:
# Tokenisasi & Ekstraksi Fitur dengan Bag of Words (BoW)
vectorizer = CountVectorizer(ngram_range=(1, 2))  # BoW dengan unigram dan bigram
X_bow = vectorizer.fit_transform(clean_df['text_akhir'])

In [30]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_bow, y_encoded, test_size=0.2, random_state=42)

In [31]:
# Membuat dan melatih model SVM
# Menggunakan GridSearch untuk tuning hyperparameter
parameters = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

grid_search = GridSearchCV(SVC(), parameters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

svm_model = grid_search.best_estimator_
print("Best parameters:", grid_search.best_params_)

Best parameters: {'C': 0.1, 'kernel': 'linear'}


In [32]:
# Prediksi pada data pelatihan dan data uji
y_pred_train_svm = svm_model.predict(X_train)
y_pred_test_svm = svm_model.predict(X_test)

# Evaluasi akurasi
accuracy_train_svm = accuracy_score(y_train, y_pred_train_svm)
accuracy_test_svm = accuracy_score(y_test, y_pred_test_svm)

# Menampilkan hasil
print('Support Vector Machine (SVM) - Accuracy (Train):', accuracy_train_svm)
print('Support Vector Machine (SVM) - Accuracy (Test):', accuracy_test_svm)

Support Vector Machine (SVM) - Accuracy (Train): 0.931936114293416
Support Vector Machine (SVM) - Accuracy (Test): 0.8991913746630728


## Skema 3

* Pelatihan: Long-Short Term Memory (LSTM)
* Ekstraksi Fitur: -
* Pembagian Data: 70/30

LSTM merupakan deep learning yang bekerja langsung dengan urutan kata (sekuens), sehingga tidak memerlukan ekstraksi fitur.

In [33]:
# Tokenisasi
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_df['text_akhir'])
sequences = tokenizer.texts_to_sequences(clean_df['text_akhir'])

In [34]:
# Padding sequence agar panjangnya sama
maxlen = 100
X_padded = pad_sequences(sequences, padding='post', maxlen=maxlen)

In [35]:
# Split data 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

In [36]:
# Membangun model LSTM
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Melatih model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 265s 645ms/step - accuracy: 0.8420 - loss: 0.5312 - val_accuracy: 0.8559 - val_loss: 0.4910
Epoch 2/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 257s 634ms/step - accuracy: 0.8581 - loss: 0.4857 - val_accuracy: 0.8559 - val_loss: 0.4901
Epoch 3/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 258s 635ms/step - accuracy: 0.8565 - loss: 0.4902 - val_accuracy: 0.8559 - val_loss: 0.4904
Epoch 4/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 258s 635ms/step - accuracy: 0.8566 - loss: 0.4918 - val_accuracy: 0.8559 - val_loss: 0.4903
Epoch 5/20
406/406 ━━━━━━━━━━━━━━━━━━━━ 257s 633ms/step - accuracy: 0.8619 - loss: 0.4768 - val_accuracy: 0.8559 - val_loss: 0.4919


In [39]:
# Prediksi
y_pred_train = np.argmax(model.predict(X_train), axis=1)
y_pred_test = np.argmax(model.predict(X_test), axis=1)

# Evaluasi
accuracy_train_lstm = accuracy_score(y_train, y_pred_train)
accuracy_test_lstm = accuracy_score(y_test, y_pred_test)

print('LSTM - Accuracy (Train):', accuracy_train_lstm)
print('LSTM - Accuracy (Test):', accuracy_test_lstm)

406/406 ━━━━━━━━━━━━━━━━━━━━ 23s 55ms/step
174/174 ━━━━━━━━━━━━━━━━━━━━ 9s 54ms/step
LSTM - Accuracy (Train): 0.8558995686999384
LSTM - Accuracy (Test): 0.855884995507637


## Menyimpan model terbaik

Dari ketiga skema analisis sentimen yang dibandingkan, model SVM memiliki akurasi testing dan evaluasi tertinggi. Yaitu	93.1% untuk akurasi training set dan 89.9% untuk testing set.

In [40]:
joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

In [41]:
joblib.dump(vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']